In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from keras_tuner import HyperModel
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Flatten
)
from keras_tuner.tuners import RandomSearch
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

enc = OneHotEncoder()
Y = enc.fit_transform(y).toarray()
X_with_bias = np.c_[np.ones((X.shape[0], 1)), X]

X_train, X_test, y_train, y_test = train_test_split(X_with_bias, Y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test, X_val, y_test, y_val = train_test_split(X_test_scaled, y_test, test_size=0.5, random_state=42)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    hp_units = hp.Int('units', min_value=5, max_value=30, step=1)
    model.add(layers.Dense(units=hp_units, activation='tanh'))
    model.add(layers.Dense(3, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Adjust the number of trials as needed
    executions_per_trial=1,
    directory='keras_tuner_logs',
    project_name='iris_hyperparameter_tuning')

tuner.search(X_train_scaled, y_train, epochs=5, validation_split=0.2)

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

eval_result = best_model.evaluate(X_test, y_test)
print("Test loss:", eval_result[0])
print("Test accuracy:", eval_result[1])
best_hyperparameters = best_hp.values
print("\nBest Hyperparameters:")
for param, value in best_hyperparameters.items():
    print(f"{param}: {value}")

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8571428656578064

Best val_accuracy So Far: 0.9047619104385376
Total elapsed time: 00h 00m 09s
1/1 [==============================] - 0s 237ms/step - loss: 0.4165 - accuracy: 0.9545
Test loss: 0.4164811968803406
Test accuracy: 0.9545454382896423

Best Hyperparameters:
units: 26
learning_rate: 0.01


In [ ]:
best_hp.values

{'units': 26, 'learning_rate': 0.01}

In [ ]:
input_size = 5
hidden_size = 26
output_size = 3

W1 = np.random.randn(input_size, hidden_size)
W2 = np.random.randn(hidden_size, output_size)
b1 = np.zeros(hidden_size)
b2 = np.zeros(output_size)

def softmax(a):
    return np.exp(a) / np.sum(np.exp(a), axis=1, keepdims=True)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, W2) + b2
    y_hat = sigmoid(z2)
    return z1, a1, z2, y_hat

def backward(X, Y, z1, a1, z2, y_hat):
    delta2 = y_hat - Y
    dW2 = np.dot(a1.T, delta2)
    db2 = np.sum(delta2, axis=0)
    delta1 = tanh_derivative(z1) * np.dot(delta2, W2.T)
    dW1 = np.dot(X.T, delta1)
    db1 = np.sum(delta1, axis=0)
    return dW1, db1, dW2, db2

def update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    return W1, b1, W2, b2

def accuracy(y_hat, Y):
    y_hat = np.argmax(y_hat, axis=1)
    return np.sum(y_hat == Y) / Y.shape[0]

epochs = 10000
learning_rate = 0.01
best_accuracy = 0.0
best_epoch = 0
best_weights = (W1, b1, W2, b2)
for epoch in range(epochs):
    z1, a1, z2, y_hat = forward(X_train)
    dW1, db1, dW2, db2 = backward(X_train, y_train, z1, a1, z2, y_hat)
    W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

    if epoch % 500 == 0:
        y_train_pred = np.argmax(forward(X_train)[-1], axis=1)
        y_test_pred = np.argmax(forward(X_test)[-1], axis=1)

        # Convert one-hot encoded labels back to integer labels
        y_train_int = np.argmax(y_train, axis=1)
        y_test_int = np.argmax(y_test, axis=1)

        print("Train Set Metrics:")
        print(classification_report(y_train_int, y_train_pred))

        print("Test Set Metrics:")
        print(classification_report(y_test_int, y_test_pred))

Train Set Metrics:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.35      1.00      0.52        37
           2       0.00      0.00      0.00        37

    accuracy                           0.35       105
   macro avg       0.12      0.33      0.17       105
weighted avg       0.12      0.35      0.18       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      0.38      0.56        13
           1       0.33      1.00      0.50         3
           2       0.00      0.00      0.00         6

    accuracy                           0.36        22
   macro avg       0.44      0.46      0.35        22
weighted avg       0.64      0.36      0.40        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.40      0.67      0.50         3
           2       0.75      0.50      0.60         6

    accuracy                           0.82        22
   macro avg       0.72      0.72      0.70        22
weighted avg       0.85      0.82      0.82        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.60      1.00      0.75         3
           2       1.00      0.67      0.80         6

    accuracy                           0.91        22
   macro avg       0.87      0.89      0.85        22
weighted avg       0.95      0.91      0.91        22



...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.33      0.50         3
           2       0.75      1.00      0.86         6

    accuracy                           0.91        22
   macro avg       0.92      0.78      0.79        22
weighted avg       0.93      0.91      0.89        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.33      0.50         3
           2       0.75      1.00      0.86         6

    accuracy                           0.91        22
   macro avg       0.92      0.78      0.79        22
weighted avg       0.93      0.91      0.89        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# now we are going to find the hyperparameter without oneHotEncoding


In [ ]:

iris = load_iris()
X = iris.data
y = iris.target

X_with_bias = np.c_[np.ones((X.shape[0], 1)), X]

X_train, X_test, y_train, y_test = train_test_split(X_with_bias, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test, X_val, y_test, y_val = train_test_split(X_test_scaled, y_test, test_size=0.5, random_state=42)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    hp_units = hp.Int('units', min_value=5, max_value=30, step=1)
    model.add(layers.Dense(units=hp_units, activation='tanh'))
    model.add(layers.Dense(3, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='keras_tuner_logs',
    project_name='iris_hyperparameter_tuning_without_OneHotEncoding')

tuner.search(X_train_scaled, y_train, epochs=5, validation_split=0.2)

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

eval_result = best_model.evaluate(X_test, y_test)
print("Test loss:", eval_result[0])
print("Test accuracy:", eval_result[1])
best_hyperparameters = best_hp.values
print("\nBest Hyperparameters:")
for param, value in best_hyperparameters.items():
    print(f"{param}: {value}")

Reloading Tuner from keras_tuner_logs/iris_hyperparameter_tuning_without_OneHotEncoding/tuner0.json


1/1 [==============================] - 0s 226ms/step - loss: 1.0032 - accuracy: 0.3636
Test loss: 1.003170132637024
Test accuracy: 0.3636363744735718

Best Hyperparameters:
units: 21
learning_rate: 1e-06


In [ ]:
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, W2) + b2
    y_hat = softmax(z2)
    return z1, a1, z2, y_hat
def backward(X, Y, z1, a1, z2, y_hat):
    # Convert integer labels to one-hot encoded labels
    Y_one_hot = np.zeros_like(y_hat)
    Y_one_hot[np.arange(len(Y)), Y] = 1

    delta2 = y_hat - Y_one_hot
    dW2 = np.dot(a1.T, delta2)
    db2 = np.sum(delta2, axis=0)
    delta1 = tanh_derivative(z1) * np.dot(delta2, W2.T)
    dW1 = np.dot(X.T, delta1)
    db1 = np.sum(delta1, axis=0)

    return dW1, db1, dW2, db2
def accuracy(y_hat, Y):
    return np.sum(np.argmax(y_hat, axis=1) == Y) / Y.shape[0]

epochs = 10000
learning_rate = 1e-06
input_size = 5
hidden_size = 21
output_size = 3

W1 = np.random.randn(input_size, hidden_size)
W2 = np.random.randn(hidden_size, output_size)
b1 = np.zeros(hidden_size)
b2 = np.zeros(output_size)

best_accuracy = 0.0
best_epoch = 0
best_weights = (W1, b1, W2, b2)
for epoch in range(epochs):
    z1, a1, z2, y_hat = forward(X_train)
    dW1, db1, dW2, db2 = backward(X_train, y_train, z1, a1, z2, y_hat)
    W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

    if epoch % 500 == 0:
        y_train_pred = np.argmax(forward(X_train)[-1], axis=1)
        y_test_pred = np.argmax(forward(X_test)[-1], axis=1)

        # Convert one-hot encoded labels back to integer labels
        if len(y_train.shape) > 1 and y_train.shape[1] > 1:
            y_train_int = np.argmax(y_train, axis=1)
            y_test_int = np.argmax(y_test, axis=1)
        else:
            y_train_int = y_train
            y_test_int = y_test
        print("Train Set Metrics:")
        print(classification_report(y_train_int, y_train_pred))

        print("Test Set Metrics:")
        print(classification_report(y_test_int, y_test_pred))

Train Set Metrics:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.00      0.00      0.00        37
           2       0.35      1.00      0.52        37

    accuracy                           0.35       105
   macro avg       0.12      0.33      0.17       105
weighted avg       0.12      0.35      0.18       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.00      0.00      0.00         3
           2       0.75      1.00      0.86         6

    accuracy                           0.82        22
   macro avg       0.56      0.64      0.59        22
weighted avg       0.75      0.82      0.78        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      0.13      0.23        31
           1       0.00      0.00      0.00        37
           2       0.37      1.00      0.54        37

    accuracy                           0.39       105
   macro avg       0.46      0.38      0.25       105
weighted avg       0.42      0.39      0.26       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         3
           2       0.75      1.00      0.86         6

    accuracy                           0.86        22
   macro avg       0.56      0.67      0.61        22
weighted avg       0.75      0.86      0.80        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      0.84      0.91        31
           1       0.00      0.00      0.00        37
           2       0.47      1.00      0.64        37

    accuracy                           0.60       105
   macro avg       0.49      0.61      0.52       105
weighted avg       0.46      0.60      0.49       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         3
           2       0.75      1.00      0.86         6

    accuracy                           0.86        22
   macro avg       0.56      0.67      0.61        22
weighted avg       0.75      0.86      0.80        22



...

Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.23      0.27      0.25        37
           2       0.10      0.08      0.09        37

    accuracy                           0.42       105
   macro avg       0.44      0.45      0.45       105
weighted avg       0.41      0.42      0.41       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         3
           2       0.75      1.00      0.86         6

    accuracy                           0.86        22
   macro avg       0.56      0.67      0.61        22
weighted avg       0.75      0.86      0.80        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.25      0.30      0.27        37
           2       0.13      0.11      0.12        37

    accuracy                           0.44       105
   macro avg       0.46      0.47      0.46       105
weighted avg       0.43      0.44      0.43       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         3
           2       0.75      1.00      0.86         6

    accuracy                           0.86        22
   macro avg       0.56      0.67      0.61        22
weighted avg       0.75      0.86      0.80        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.27      0.32      0.29        37
           2       0.14      0.11      0.12        37

    accuracy                           0.45       105
   macro avg       0.47      0.48      0.47       105
weighted avg       0.44      0.45      0.44       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         3
           2       0.71      0.83      0.77         6

    accuracy                           0.82        22
   macro avg       0.55      0.61      0.58        22
weighted avg       0.74      0.82      0.78        22

Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.30      0.35      0.32        37
           2       

# Now let see what will be the accuracy if there is no bias column

In [ ]:

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test, X_val, y_test, y_val = train_test_split(X_test_scaled, y_test, test_size=0.5, random_state=42)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    hp_units = hp.Int('units', min_value=5, max_value=30, step=1)
    model.add(layers.Dense(units=hp_units, activation='tanh'))
    model.add(layers.Dense(3, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='keras_tuner_logs',
    project_name='NO_bias_iris_hyperparameter_tuning_without_OneHotEncoding')

tuner.search(X_train_scaled, y_train, epochs=5, validation_split=0.2)

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

eval_result = best_model.evaluate(X_test, y_test)
print("Test loss:", eval_result[0])
print("Test accuracy:", eval_result[1])
best_hyperparameters = best_hp.values
print("\nBest Hyperparameters:")
for param, value in best_hyperparameters.items():
    print(f"{param}: {value}")

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.0476190485060215

Best val_accuracy So Far: 0.523809552192688
Total elapsed time: 00h 00m 09s
1/1 [==============================] - 0s 218ms/step - loss: 0.9638 - accuracy: 0.5455
Test loss: 0.9638338685035706
Test accuracy: 0.5454545617103577

Best Hyperparameters:
units: 9
learning_rate: 0.001


In [ ]:

epochs = 10000
learning_rate = 0.001
input_size = 4
hidden_size = 9
output_size = 3

W1 = np.random.randn(input_size, hidden_size)
W2 = np.random.randn(hidden_size, output_size)
b1 = np.zeros(hidden_size)
b2 = np.zeros(output_size)

best_accuracy = 0.0
best_epoch = 0
best_weights = (W1, b1, W2, b2)
for epoch in range(epochs):
    z1, a1, z2, y_hat = forward(X_train)
    dW1, db1, dW2, db2 = backward(X_train, y_train, z1, a1, z2, y_hat)
    W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

    if epoch % 500 == 0:
        y_train_pred = np.argmax(forward(X_train)[-1], axis=1)
        y_test_pred = np.argmax(forward(X_test)[-1], axis=1)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_epoch = epoch
            best_weights = (W1, b1, W2, b2)
        print("Train Set Metrics:")
        print(f"Accuracy: {accuracy_score(y_train, y_train_pred):.3f}")
        print(f"Precision: {precision_score(y_train, y_train_pred, average='weighted'):.3f}")
        print(f"Recall: {recall_score(y_train, y_train_pred, average='weighted'):.3f}")
        print(f"F1-score: {f1_score(y_train, y_train_pred, average='weighted'):.3f}")
        print(f"Confusion Matrix: \n{confusion_matrix(y_train, y_train_pred)}")

        # Compute metrics for test set
        print("Test Set Metrics:")
        print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")
        print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted'):.3f}")
        print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted'):.3f}")
        print(f"F1-score: {f1_score(y_test, y_test_pred, average='weighted'):.3f}")
        print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_test_pred)}")
print(f"Best Test Accuracy: {best_accuracy} at Epoch: {best_epoch}")
W1, b1, W2, b2 = best_weights

Train Set Metrics:
Accuracy: 0.581
Precision: 0.438
Recall: 0.581
F1-score: 0.473
Confusion Matrix: 
[[24  0  7]
 [ 2  0 35]
 [ 0  0 37]]
Test Set Metrics:
Accuracy: 0.273
Precision: 0.427
Recall: 0.273
F1-score: 0.291
Confusion Matrix: 
[[3 2 8]
 [1 1 1]
 [1 3 2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
Accuracy: 0.629
Precision: 0.452
Recall: 0.629
F1-score: 0.511
Confusion Matrix: 
[[29  0  2]
 [ 2  0 35]
 [ 0  0 37]]
Test Set Metrics:
Accuracy: 0.273
Precision: 0.427
Recall: 0.273
F1-score: 0.292
Confusion Matrix: 
[[3 1 9]
 [1 1 1]
 [1 3 2]]
Train Set Metrics:
Accuracy: 0.638


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.462
Recall: 0.638
F1-score: 0.521
Confusion Matrix: 
[[30  0  1]
 [ 1  0 36]
 [ 0  0 37]]
Test Set Metrics:
Accuracy: 0.273
Precision: 0.431
Recall: 0.273
F1-score: 0.293
Confusion Matrix: 
[[ 3  0 10]
 [ 1  1  1]
 [ 1  3  2]]
Train Set Metrics:
Accuracy: 0.638
Precision: 0.462
Recall: 0.638
F1-score: 0.521
Confusion Matrix: 
[[30  0  1]
 [ 1  0 36]
 [ 0  0 37]]

...

Train Set Metrics:
Accuracy: 0.648
Precision: 0.465
Recall: 0.648
F1-score: 0.528
Confusion Matrix: 
[[31  0  0]
 [ 1  0 36]
 [ 0  0 37]]
Test Set Metrics:
Accuracy: 0.182
Precision: 0.377
Recall: 0.182
F1-score: 0.227
Confusion Matrix: 
[[ 3  0 10]
 [ 1  1  1]
 [ 1  5  0]]
Train Set Metrics:
Accuracy: 0.648
Precision: 0.465
Recall: 0.648
F1-score: 0.528
Confusion Matrix: 
[[31  0  0]
 [ 1  0 36]
 [ 0  0 37]]
Test Set Metrics:
Accuracy: 0.182
Precision: 0.377
Recall: 0.182
F1-score: 0.227
Confusion Matrix: 
[[ 3  0 10]
 [ 1  1  1]
 [ 1  5  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
Accuracy: 0.648
Precision: 0.465
Recall: 0.648
F1-score: 0.528
Confusion Matrix: 
[[31  0  0]
 [ 1  0 36]
 [ 0  0 37]]
Test Set Metrics:
Accuracy: 0.227
Precision: 0.417
Recall: 0.227
F1-score: 0.279
Confusion Matrix: 
[[4 0 9]
 [1 1 1]
 [1 5 0]]
Train Set Metrics:
Accuracy: 0.610
Precision: 0.461
Recall: 0.610
F1-score: 0.513
Confusion Matrix: 
[[31  0  0]
 [ 0  0 37]
 [ 0  4 33]]
Test Set Metrics:
Accuracy: 0.227
Precision: 0.417
Recall: 0.227
F1-score: 0.279
Confusion Matrix: 
[[4 0 9]
 [1 1 1]
 [1 5 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
Accuracy: 0.619
Precision: 0.581
Recall: 0.619
F1-score: 0.555
Confusion Matrix: 
[[31  0  0]
 [ 0  3 34]
 [ 0  6 31]]
Test Set Metrics:
Accuracy: 0.227
Precision: 0.417
Recall: 0.227
F1-score: 0.279
Confusion Matrix: 
[[4 0 9]
 [1 1 1]
 [1 5 0]]
Train Set Metrics:
Accuracy: 0.638
Precision: 0.629
Recall: 0.638
F1-score: 0.587
Confusion Matrix: 
[[31  0  0]
 [ 0  5 32]
 [ 0  6 31]]
Test Set Metrics:
Accuracy: 0.227
Precision: 0.417
Recall: 0.227
F1-score: 0.279
Confusion Matrix: 
[[4 0 9]
 [1 1 1]
 [1 5 0]]
Train Set Metrics:
Accuracy: 0.657
Precision: 0.661
Recall: 0.657
F1-score: 0.630
Confusion Matrix: 
[[31  0  0]
 [ 0  9 28]
 [ 0  8 29]]
Test Set Metrics:
Accuracy: 0.227
Precision: 0.417
Recall: 0.227
F1-score: 0.279
Confusion Matrix: 
[[4 0 9]
 [1 1 1]
 [1 5 0]]
Train Set Metrics:
Accuracy: 0.714
Precision: 0.725
Recall: 0.714
F1-score: 0.704
Confusion Matrix: 
[[31  0  0]
 [ 0 15 22]
 [ 0  8 29]]
Test Set Metrics:
Accuracy: 0.227
Precision: 0.417
Recall: 0.227

## with oneHoTEncoding without bias

In [ ]:

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

enc = OneHotEncoder()
Y = enc.fit_transform(y).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test, X_val, y_test, y_val = train_test_split(X_test_scaled, y_test, test_size=0.5, random_state=42)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    hp_units = hp.Int('units', min_value=5, max_value=30, step=1)
    model.add(layers.Dense(units=hp_units, activation='tanh'))
    model.add(layers.Dense(3, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='keras_tuner_logs',
    project_name='NO_bias_iris_hyperparameter_tuning')

tuner.search(X_train_scaled, y_train, epochs=5, validation_split=0.2)

best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

eval_result = best_model.evaluate(X_test, y_test)
print("Test loss:", eval_result[0])
print("Test accuracy:", eval_result[1])
best_hyperparameters = best_hp.values
print("\nBest Hyperparameters:")
for param, value in best_hyperparameters.items():
    print(f"{param}: {value}")

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.095238097012043

Best val_accuracy So Far: 0.8571428656578064
Total elapsed time: 00h 00m 09s
1/1 [==============================] - 0s 232ms/step - loss: 0.2324 - accuracy: 0.9091
Test loss: 0.2323707938194275
Test accuracy: 0.9090909361839294

Best Hyperparameters:
units: 27
learning_rate: 0.01


In [ ]:
input_size = 4
hidden_size = 27
output_size = 3

W1 = np.random.randn(input_size, hidden_size)
W2 = np.random.randn(hidden_size, output_size)
b1 = np.zeros(hidden_size)
b2 = np.zeros(output_size)

def softmax(a):
    return np.exp(a) / np.sum(np.exp(a), axis=1, keepdims=True)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, W2) + b2
    y_hat = sigmoid(z2)
    return z1, a1, z2, y_hat

def backward(X, Y, z1, a1, z2, y_hat):
    delta2 = y_hat - Y
    dW2 = np.dot(a1.T, delta2)
    db2 = np.sum(delta2, axis=0)
    delta1 = tanh_derivative(z1) * np.dot(delta2, W2.T)
    dW1 = np.dot(X.T, delta1)
    db1 = np.sum(delta1, axis=0)
    return dW1, db1, dW2, db2

def update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    return W1, b1, W2, b2

def accuracy(y_hat, Y):
    return np.sum(np.argmax(y_hat, axis=1) == np.argmax(Y, axis=1)) / Y.shape[0]


epochs = 10000
learning_rate = 0.01
best_accuracy = 0.0
best_epoch = 0
best_weights = (W1, b1, W2, b2)
for epoch in range(epochs):
    z1, a1, z2, y_hat = forward(X_train)
    dW1, db1, dW2, db2 = backward(X_train, y_train, z1, a1, z2, y_hat)
    W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

    if epoch % 500 == 0:
        y_train_pred = np.argmax(forward(X_train)[-1], axis=1)
        y_test_pred = np.argmax(forward(X_test)[-1], axis=1)

        # Convert one-hot encoded labels back to integer labels
        y_train_int = np.argmax(y_train, axis=1)
        y_test_int = np.argmax(y_test, axis=1)

        print("Train Set Metrics:")
        print(classification_report(y_train_int, y_train_pred))

        print("Test Set Metrics:")
        print(classification_report(y_test_int, y_test_pred))

Train Set Metrics:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.35      1.00      0.52        37
           2       0.00      0.00      0.00        37

    accuracy                           0.35       105
   macro avg       0.12      0.33      0.17       105
weighted avg       0.12      0.35      0.18       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.25      0.67      0.36         3
           2       0.00      0.00      0.00         6

    accuracy                           0.09        22
   macro avg       0.08      0.22      0.12        22
weighted avg       0.03      0.09      0.05        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.33      0.67      0.44         3
           2       0.67      0.33      0.44         6

    accuracy                           0.77        22
   macro avg       0.67      0.67      0.63        22
weighted avg       0.82      0.77      0.77        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      0.03      0.05        37
           2       0.51      1.00      0.67        37

    accuracy                           0.66       105
   macro avg       0.84      0.68      0.58       105
weighted avg       0.83      0.66      0.55       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.43      1.00      0.60         3
           2       1.00      0.33      0.50         6

    accuracy                           0.82        22
   macro avg       0.81      0.78      0.70        22
weighted avg       0.92      0.82      0.81        22

Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.83      0.14      0.23        37
           2       

...

Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.33      0.50         3
           2       0.75      1.00      0.86         6

    accuracy                           0.91        22
   macro avg       0.92      0.78      0.79        22
weighted avg       0.93      0.91      0.89        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.00      0.00      0.00        37
           2       0.50      1.00      0.67        37

    accuracy                           0.65       105
   macro avg       0.50      0.67      0.56       105
weighted avg       0.47      0.65      0.53       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.33      0.50         3
           2       0.75      1.00      0.86         6

    accuracy                           0.91        22
   macro avg       0.92      0.78      0.79        22
weighted avg       0.93      0.91      0.89        22



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      0.03      0.05        37
           2       0.51      1.00      0.67        37

    accuracy                           0.66       105
   macro avg       0.84      0.68      0.58       105
weighted avg       0.83      0.66      0.55       105

Test Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.33      0.50         3
           2       0.75      1.00      0.86         6

    accuracy                           0.91        22
   macro avg       0.92      0.78      0.79        22
weighted avg       0.93      0.91      0.89        22

Train Set Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      0.03      0.05        37
           2       